In [1]:
import csv
import subprocess
from pymongo import MongoClient
import time
import os
from datetime import datetime, timedelta 
import schedule

In [10]:
def save_mongo(filepath):
    try:
        # Koneksi ke MongoDB
        with MongoClient('mongodb://localhost:27017/') as client:
            db = client.db_analisis_sentimen
            tweets_collection = db.tweets

            # Membuka file CSV
            with open(filepath, 'r', newline='', encoding='utf-8') as csvfile:
                reader = csv.DictReader(csvfile)

                for tweet in reader:
                    # Pastikan "id_str" ada dan tidak kosong
                    if tweet.get("id_str") and tweet["id_str"] != "id_str":
                        existing_tweet = tweets_collection.find_one({"full_text": tweet.get("full_text")})

                        if not existing_tweet:
                            tweet_to_save = {
                                "conversation_id_str": tweet.get("conversation_id_str"), 
                                "created_at": tweet.get("created_at"), 
                                "favorite_count": tweet.get("favorite_count"),
                                "full_text": tweet.get("full_text"),
                                "id_str": tweet.get("id_str"),  
                                "username": tweet.get("username"),
                                "tweet_url": tweet.get("tweet_url"),
                                "image_url": tweet.get("image_url"),  
                                "in_reply_to_screen_name": tweet.get("in_reply_to_screen_name"),  
                                "lang": tweet.get("lang"), 
                                "location": tweet.get("location"),  
                                "quote_count": tweet.get("quote_count"),  
                                "reply_count": tweet.get("reply_count"),  
                                "retweet_count": tweet.get("retweet_count"),  
                                "user_id_str": tweet.get("user_id_str"),  
                            }
                            tweets_collection.insert_one(tweet_to_save)

        print("Data berhasil disimpan ke MongoDB.")
    
    except Exception as e:
        print(f"Terjadi kesalahan: {e}")

In [2]:
current_date = datetime.now().date()
yesterday = current_date - timedelta(days=1)

date_format = yesterday.strftime("%Y-%m-%d")
filename = 'tweets.csv'
limit = 50

search_keyword = f'unnes since:{date_format} until:{current_date}'

In [7]:
npx_command = f'npx --yes tweet-harvest@latest -o "{filename}" -s "{search_keyword}" -l {limit} --token "4542b47c1a5e8af47d4968ec7a1d1f6360c3c091"'

In [8]:
result = subprocess.run(npx_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, encoding="utf-8")
print("Subprocess Output:", result.stdout)
print("Subprocess Errors:", result.stderr)

Subprocess Output: Tweet Harvest [v2.6.1]

Research by Helmi Satria
Use it for Educational Purposes only!

This script uses Chromium Browser to crawl data from Twitter with your Twitter auth token.
Please enter your Twitter auth token when prompted.

Note: Keep your access token secret! Don't share it with anyone else.
Note: This script only runs on your local device.


Opening twitter search page...


Filling in keywords: unnes since:2025-03-13 until:2025-03-14


-- Scrolling... (1)Created new directory: \\?\c:\Users\Ahmad Dani\codingz\SENTIS\app\crawling\tweets-data


Your tweets saved to: c:\Users\Ahmad Dani\codingz\SENTIS\app\crawling\tweets-data\tweets.csv
Total tweets saved: 19

-- Scrolling... (1) (2)

Your tweets saved to: c:\Users\Ahmad Dani\codingz\SENTIS\app\crawling\tweets-data\tweets.csv
Total tweets saved: 39


Your tweets saved to: c:\Users\Ahmad Dani\codingz\SENTIS\app\crawling\tweets-data\tweets.csv
Total tweets saved: 59
Got 59 tweets, done scrolling...

Subprocess Er

In [11]:
file_path = os.path.join('tweets-data', 'tweets.csv')
save_mongo(file_path)

Data berhasil disimpan ke MongoDB.
